### Instruction

To obtain the dataset Internet used for algorithms comparison:

1) Download `kdd_internet_usage.arff` file from http://www.cs.odu.edu/~mukka/cs795sum10dm/datasets/uci-20070111/nominal/kdd_internet_usage.arff.

2) Put it to the same directory as this notebook.

3) Run all the cells of this notebook successively to produce files for training and testing.

In [1]:
resulting_train_filename = "train"
resulting_test_filename = "test"

### Preparing the data

In [2]:
import pandas as pd
import re
import scipy.io.arff

In [3]:
with open("kdd_internet_usage.arff", "rb") as fin:
    data, meta = scipy.io.arff.loadarff(fin)
    data = pd.DataFrame(data)

In [4]:
data.head()

,Actual_Time,Age,Community_Building,Community_Membership_Family,Community_Membership_Hobbies,Community_Membership_None,Community_Membership_Other,Community_Membership_Political,Community_Membership_Professional,Community_Membership_Religious,...,Web_Page_Creation,Who_Pays_for_Access_Dont_Know,Who_Pays_for_Access_Other,Who_Pays_for_Access_Parents,Who_Pays_for_Access_School,Who_Pays_for_Access_Self,Who_Pays_for_Access_Work,Willingness_to_Pay_Fees,Years_on_Internet,who
0,Consultant,41,Equally,0,0,1,0,0,0,0,...,Yes,0,0,0,0,1,0,Other_sources,1-3_yr,93819
1,College_Student,28,Equally,0,0,0,0,0,0,0,...,No,0,0,0,0,1,0,Already_paying,Under_6_mo,95708
2,Other,25,More,1,1,0,0,0,1,0,...,Yes,0,0,0,0,1,1,Other_sources,1-3_yr,97218
3,Salesperson,28,More,0,0,0,1,0,0,0,...,Yes,0,0,0,0,1,0,Already_paying,1-3_yr,91627
4,K-12_Student,17,More,0,0,0,0,1,1,0,...,Yes,0,0,0,0,1,0,Already_paying,1-3_yr,49906


In [5]:
target = data["Who_Pays_for_Access_Work"].apply(lambda x: 1 if x == '0' else -1)
data.drop(["Who_Pays_for_Access_Work", "Willingness_to_Pay_Fees", "Years_on_Internet", "who"], axis=1, inplace=True)

In [6]:
data.shape

(10108, 68)

### Preparing train/test split

In [7]:
train_idx = pd.read_csv("stratified_train_idx.txt", header=None)
test_idx = pd.read_csv("stratified_test_idx.txt", header=None)

In [8]:
Xtrain = data.iloc[train_idx[0]]
Ytrain = target.iloc[train_idx[0]]
Xtest = data.iloc[test_idx[0]]
Ytest = target.iloc[test_idx[0]]

In [9]:
# creating file with features
def prepare_pool(data, labels, filename):
    X = data.values
    y = labels.values
    with open(filename, "w") as fout:
        for i in range(data.shape[0]):
            fout.write(str(y[i]) + "\t" + "\t".join(map(str, X[i])) + "\n")

In [10]:
prepare_pool(Xtrain, Ytrain, resulting_train_filename)
prepare_pool(Xtest, Ytest, resulting_test_filename)

In [11]:
categorical_features = {0, 1, 2, 11, 12, 18, 19, 20, 21, 31, 32, 33, 34, 36, 37, 38, 39, 59, 60, 61, 62}
with open(resulting_train_filename + '.cd', 'w') as fout:
    fout.write('0\tTarget\n')
    for cat_f_id in sorted(categorical_features):
        fout.write('{}\tCateg\n'.format(cat_f_id + 1))